note to me:   
läs in data

>bygg X och Y  

>importera klassen  

>fit  

>skriv ut resultat

läsa in data

bygga designmatris X

skapa Y

skapa modellen

köra fit

skriva ut resultat

**numpy** för funktioner till matriser, linjär algebra, statistik, OLS-formeln senare  
**housing_data =**  Bygger numpy struktur i minnet  
**f"{directory_data}/housing.csv"** fulla sökvägen till filen  
**np.genfromtxt** öppnar filen, läser den rad för rad delar upp den vid:   
>- kommatecken (delimiter=",")  
>- Tolkar första rad som kolumnnamn, skapar strukturerad array, behandlar raden som metadat och det går att hämta specifik kolumn (names=True)  
>- tolkar siffrorna, om det är int, float eller text (dtype= None)  

tänk:  
>genfromtxt(  
    >>filens adress,  
>>hur den ska delas upp,  
>>om första raden är namn,  
>>hur typer ska tolkas  
)


In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [2]:
import numpy as np

directory_data = "../data"   #flexible directory for data files, can be changed to fit the user's file structure

housing_data = np.genfromtxt(f"{directory_data}/housing.csv",
delimiter= ",",
names= True,
dtype= None,
encoding= "utf-8"
)

housing_data.shape

(20640,)

Kolla datan

In [3]:
print(housing_data.shape) 
print(housing_data.dtype.names)
print(housing_data[:3])


(20640,)
('longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity')
[(-122.23, 37.88, 41.,  880.,  129.,  322.,  126., 8.3252, 452600., 'NEAR BAY')
 (-122.22, 37.86, 21., 7099., 1106., 2401., 1138., 8.3014, 358500., 'NEAR BAY')
 (-122.24, 37.85, 52., 1467.,  190.,  496.,  177., 7.2574, 352100., 'NEAR BAY')]


För kategorin **ocean_proximity** som jag tänker är en kategori som påvrekar priset kraftigt:  
kolla vilka vilka unika kategorier som finns som  laternativ:


In [4]:
np.unique(housing_data["ocean_proximity"])

array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype='<U10')

Använder dummy-variables för att kunna använda i en regression (kräver numerisk representation). en kategori får vara refernskategori som de övriga jämförs mot (k-1): 
I detta fall blir "inland" min referenskategori mot de övriga kategorierna.
Skapar funktion för att ta en kategorisk kolumn, välja en referenskategori, skapa dummyvariabel för övriga kategorier, returnera en ny matris med dessa summykategorier.

In [5]:
def create_dummies(column, reference):
    categories = np.unique(column)  # Get unique categories in the column
    dummy_columns = []  # Initialize a list to hold the dummy columns
    used_categories = []  # Keep track of categories that have been used for dummy variables    
    for category in categories:
        if category != reference:
            dummy = (column == category).astype(int)  # Create a dummy variable for the category
            dummy_columns.append(dummy)  # Append the dummy variable to the list
            used_categories.append(category)  # Add the category to the list of used categories
    return np.column_stack(dummy_columns), used_categories  # Stack the dummy columns together and return as a new array

print(create_dummies(housing_data["ocean_proximity"], "INLAND")[:5]) # Test the create_dummies function on the "ocean_proximity" column, using "INLAND" as the reference category. The output will show the first 5 rows of the dummy variables created for the other categories in "ocean_proximity".

(array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       ...,
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], shape=(20640, 4)), [np.str_('<1H OCEAN'), np.str_('ISLAND'), np.str_('NEAR BAY'), np.str_('NEAR OCEAN')])


kolla vilka kategorier matrisen motsvarar ([0, 0, 0, 0] = "inland" ->referenskaetgorin):

In [6]:
dummies, labels = create_dummies(housing_data["ocean_proximity"], "INLAND") # Create dummy variables for the "ocean_proximity" column, using "INLAND" as the reference category. The resulting dummy variables are stored in the 'dummies' variable, and the corresponding category labels are stored in the 'labels' variable.
print(labels) # Print the category labels that were used to create the dummy variables. This will show which categories in "ocean_proximity" were converted into dummy variables, excluding the reference category "INLAND".

[np.str_('<1H OCEAN'), np.str_('ISLAND'), np.str_('NEAR BAY'), np.str_('NEAR OCEAN')]


**Bygg designmatris**  
OLS behöver X och Y för att kunna beräkna b.  
skapa responsvektor y:

In [7]:
y = housing_data["median_house_value"].reshape(-1, 1) # Extract the "median_house_value" column from the housing_data array and store it in the variable 'y'. This variable will be used as the target variable for regression analysis.



Skapa x-matris ( intercept, median_income, population och dummykolumner)


In [8]:
ocean = housing_data["ocean_proximity"] # Extract the "ocean_proximity" column from the housing_data array and store it in the variable 'ocean'. This variable will be used as a feature for regression analysis, and may need to be converted into dummy variables for modeling purposes.
categories = np.unique(ocean) # Get the unique categories in the 'ocean' variable, which represents the different levels of proximity to the ocean. This will help identify the distinct categories that may need to be encoded as dummy variables for regression analysis.
dummy_cols = [] # Initialize an empty list to hold the dummy variable columns that will be created for the 'ocean' variable.
dummy_labels = []#COesponding to the dummy variable columns that will be created for the 'ocean' variable. These labels will indicate which category each dummy variable represents.

for category in categories: # Iterate through each unique category in the 'ocean' variable to create dummy variables for regression analysis.
    if category != "INLAND": # Check if the current category is not "INLAND", which will be used as the reference category for creating dummy variables. Only categories other than "INLAND" will be converted into dummy variables.
        dummy_cols.append((ocean == category).astype(int)) # Create a dummy variable for the current category by comparing the 'ocean' variable to the category and converting the boolean result to an integer (1 for True, 0 for False). Append this dummy variable to the 'dummy_cols' list.
        dummy_labels.append(category) # Append the category name to the 'dummy_labels' list to keep track of which category each dummy variable represents.
dummies = np.column_stack(dummy_cols) # Stack the list of dummy variable columns horizontally to create a new array of dummy variables that can be used in regression analysis. Each column in this new array corresponds to a different category from the 'ocean' variable, excluding the reference category "INLAND".
median_income = housing_data["median_income"].reshape(-1, 1) # Extract the "median_income" column from the housing_data array and store it in the variable 'median_income'. This variable will be used as a feature for regression analysis, representing the median income of households in the dataset.
population = housing_data["population"].reshape(-1, 1) # Extract the "population" column from the housing_data array and store it in the variable 'population'. This variable will be used as a feature for regression analysis, representing the population of households in the dataset.

n = len(median_income) # Calculate the number of samples in the "median_income" variable by taking the length of the array. This will be used to determine the number of rows in the feature matrix for regression analysis.
intercept = np.ones((n, 1)) # Create an intercept term (a column of ones) with the same number of rows as the "median_income" variable. This intercept term will be included in the feature matrix for regression analysis to account for the baseline level of the target variable when all features are zero.
X = np.hstack((intercept, median_income, population, dummies)) # Stack the intercept term, "median_income", "population", and the dummy variables together horizontally to create the feature matrix 'X' for regression analysis. Each column in 'X' corresponds to a different feature, and each row corresponds to a different sample in the dataset.
labels = ["Intercept", "Median_income", "Population"] + list(dummy_labels) # Create a list of labels for the features in the regression model, starting with "Intercept", followed by "Median_income" and "Population", and then appending the labels corresponding to the dummy variables created for the 'ocean' variable. This list will help identify which feature corresponds to each column in the feature matrix 'X' when interpreting the regression results.

print(X.shape)
print(labels)

(20640, 7)
['Intercept', 'Median_income', 'Population', np.str_('<1H OCEAN'), np.str_('ISLAND'), np.str_('NEAR BAY'), np.str_('NEAR OCEAN')]


x.shape: 7 kolumner/parametrar i modellen, 20640 rader/bostäder
y.shape: 1kolumn/pris, 20640 bostäder

In [9]:
print(X.shape)
print(y.shape)

(20640, 7)
(20640, 1)


Imprtera Linear Regression klass

In [10]:
from src.linear_regression import LinearRegression # Import the LinearRegression class from the linear_regression module, which will be used to perform linear regression analysis on the dataset.
model = LinearRegression() # Create an instance of the LinearRegression class and store it in the variable 'model'. This instance will be used to fit the linear regression model to the data and make predictions.
model.fit(X,y)

print("Beta: ", model.beta) # Print the estimated coefficients (beta values) of the fitted linear regression model. These coefficients represent the relationship between each feature in the feature matrix 'X' and the target variable 'y', indicating how much the target variable is expected to change for a one-unit change in each feature, while holding other features constant.
print("Variance estimate: ", model.estimate_variance()) # Print the estimated variance of the residuals (errors) from the fitted linear regression model. This variance estimate provides information about the variability of the errors around the fitted regression line, which can be used to assess the goodness of fit of the model and to construct confidence intervals for the estimated coefficients.
print("Standard Deviation: ", model.standard_deviation()) # Print the standard deviation of the residuals (errors) from the fitted linear regression model. The standard deviation is a measure of the average magnitude of the errors, and it provides insight into how well the model fits the data. A smaller standard deviation indicates that the predictions are closer to the actual values on average, while a larger standard deviation suggests that there is more variability in the errors.
print("RMSE: ", model.rmse()) # Print the Root Mean Squared Error (RMSE) of the fitted linear regression model. RMSE is a measure of the average magnitude of the errors between the predicted values and the actual values of the target variable 'y'. A lower RMSE indicates a better fit of the model to the data, as it means that the predictions are closer to the actual values on average.

Beta:  [[ 1.01700574e+04]
 [ 3.70148631e+04]
 [-2.97993489e+00]
 [ 7.78464826e+04]
 [ 2.70676208e+05]
 [ 9.82497621e+04]
 [ 9.50252015e+04]]
Variance estimate:  5476310763.736962
Standard Deviation:  74002.09972518998
RMSE:  73989.54985536571
